# Enrichment of regulons from Regulons_by_strain

In [1]:
import os
import sys
from importlib import reload

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.insert(0,module_path)

import pandas as pd
from IPython.display import display
pd.set_option('max_colwidth', 70)
pd.set_option('display.max_rows', 500)
import string

from orangecontrib.bioinformatics.geneset.__init__ import list_all

import jupyter_functions as jf
reload(jf)
import jupyter_functions as jf
from functionsDENet import loadPickle, savePickle
import stages_DE.enrichment_library as enr
reload(enr)
import stages_DE.enrichment_library as enr

In [2]:
#Path for expression data (mergedGenes_RPKUM.tsv)
dataPath='/home/karin/Documents/timeTrajectories/data/RPKUM/combined/'
#Path for regulons
pathRegulons='/home/karin/Documents/timeTrajectories/data/regulons/by_strain/kN300_mean0std1_log/'

## Used gene sets
Use gene sets that do not have less than 5 or more than 500 genes. Use all ontologies.

In [18]:
# Get all gene sets
gene_sets=list(list_all(organism='44689'))
GENE_SETS_ONTOLOGY=enr.get_gene_sets(gene_set_names=gene_sets, go_slims=True,set_sizes=(5,500))

In [13]:
# For conversion of gene names to EID
genes = pd.read_csv(dataPath + 'mergedGenes_RPKUM.tsv', sep='\t', index_col=0)
all_gene_names= genes[(genes != 0).any(axis=1)].index
NAME_EID=enr.name_genes_entrez(gene_names=all_gene_names, key_entrez=False)
ALL_GENE_NAMES_EID=enr.convert_EID(genes=all_gene_names, name_EID=NAME_EID)

In [14]:
def group_diff_enrichment(data:pd.DataFrame,group:str,padj:float=0.25,min_overlap:int=None,
                          use_annotated_genes:bool=False):
    #Displays only gene sets that have overlap with query greater or equal to min_overlap 
    #For p value and padj calculation uses alll that have overlap >=1 } from gene_set_enrichment
    """
    :param use_annotated_genes: if True use for reference and query  only genes that have at 
    least one gene set annotation
    """
    selected=list(data[data['Cluster']==group].index)
    query_EID=enr.convert_EID(genes=selected, name_EID=NAME_EID)
    print('***  '+group+' selected:',len(selected),'with EID:',len(query_EID))
    
    reference_gene_eids=ALL_GENE_NAMES_EID.copy()
    query_eids=query_EID.copy()
    
    if use_annotated_genes:
        gene_sets_genes=set()
        for gene_set_name, gene_sets in GENE_SETS_ONTOLOGY.items():
            for gene_set in gene_sets:
                gene_sets_genes.update(gene_set.genes)
        reference_gene_eids=set(reference_gene_eids) & gene_sets_genes
        query_eids=set(query_eids) & gene_sets_genes
        print('Ratio of genes annotated with a gene set in reference',
              round(len(reference_gene_eids)/len(ALL_GENE_NAMES_EID),2),
             'and query',round(len(query_eids)/len(query_EID),2))
    
    query_in_enriched=set()
    result=None
    if len(query_eids) > 0:
        enrichment=enr.gene_set_enrichment(query_eids, reference_EID=reference_gene_eids, 
                                                padj_threshold=padj,min_overlap=min_overlap,
                                                gene_sets_ontology=GENE_SETS_ONTOLOGY)
        if len(enrichment)>0:
            enrichment_display=list()
            enrichment= sorted(enrichment, key=lambda data: data.padj)
            for enriched in enrichment:
                query_in_enriched.update(enriched.gene_set.genes & query_eids)
                enrichment_display.append({'Gene set':enriched.gene_set.name,'Ontology':enriched.ontology,
                                           'FDR':"{:.2e}".format(enriched.padj),'N in query':enriched.in_query})
            result=pd.DataFrame(enrichment_display)
    print('Enrichment at FDR: '+str(padj)+' and min query - gene set overlap',str(min_overlap))
    print('N query genes in displayed gene sets:',len(query_in_enriched),'out of', len(query_eids),
          'query genes used for enrichment calculation.')
    display(result)
    print('\n')
    return result

## Enrichment reports

In [16]:
save_enrichment=False

### Enrichment for expression_minExpressed0.990.1Strains1Min1Max18_clustersLouvain0.4minmaxNologPCA30kN30.pdf

In [15]:
regulons=pd.read_table(pathRegulons+'mergedGenes_minExpressed0.990.1Strains1Min1Max18_clustersLouvain0.4minmaxNologPCA30kN30.tab',index_col=0)
for group in range(1,len(regulons['Cluster'].unique())+1):
    enriched=group_diff_enrichment(data=regulons,group='C'+str(group),min_overlap=2)
    if save_enrichment and enriched is not None:
        enriched.to_csv(pathRegulons+'enrichment/minExpressed0.990.1Strains1Min1Max18_clustersLouvain0.4minmaxNologPCA30kN30_enrichment_cluster'+str(group)+'.tsv', sep='\t',index=False)

***  C1 selected: 30 with EID: 30
Enrichment at FDR: 0.25 and min query - gene set overlap 2
N query genes in displayed gene sets: 20 out of 30 query genes used for enrichment calculation.


,Gene set,Ontology,FDR,N in query
0,Ribosome biogenesis in eukaryotes,"(KEGG, Pathways)",3.61e-19,12
1,nucleolus,"(GO, cellular_component)",4.81e-17,11
2,ribosome biogenesis,"(GO, biological_process)",5.97e-16,10
3,ribonucleoprotein complex assembly,"(GO, biological_process)",3.39e-07,4
4,RNA binding,"(GO, molecular_function)",8.74e-06,6
5,protein-containing complex assembly,"(GO, biological_process)",7.59e-04,4
6,rRNA binding,"(GO, molecular_function)",1.00e-03,2
7,cellular component assembly,"(GO, biological_process)",2.69e-03,4
8,helicase activity,"(GO, molecular_function)",1.20e-02,2
9,methyltransferase activity,"(GO, molecular_function)",1.30e-02,2




***  C2 selected: 66 with EID: 65
Enrichment at FDR: 0.25 and min query - gene set overlap 2
N query genes in displayed gene sets: 34 out of 65 query genes used for enrichment calculation.


,Gene set,Ontology,FDR,N in query
0,Oxidative phosphorylation,"(KEGG, Pathways)",6.91e-10,9
1,Metabolic pathways,"(KEGG, Pathways)",3.34e-08,19
2,mitochondrion,"(GO, cellular_component)",8.58e-05,9
3,Citrate cycle (TCA cycle),"(KEGG, Pathways)",3.86e-04,4
4,Phagosome,"(KEGG, Pathways)",2.80e-03,4
5,generation of precursor metabolites and energy,"(GO, biological_process)",1.06e-02,4
6,cytoplasmic vesicle,"(GO, cellular_component)",1.06e-02,6
7,protein folding,"(GO, biological_process)",1.06e-02,3
8,aberrant cellular response to stress,"(Dictybase, Phenotypes)",1.44e-02,2
9,Ribosome,"(KEGG, Pathways)",1.50e-02,4




***  C3 selected: 68 with EID: 68
Enrichment at FDR: 0.25 and min query - gene set overlap 2
N query genes in displayed gene sets: 62 out of 68 query genes used for enrichment calculation.


...

,Gene set,Ontology,FDR,N in query
0,cell adhesion,"(GO, biological_process)",8.13e-02,3
1,extracellular region,"(GO, cellular_component)",8.13e-02,4
2,hydrolase activity,"(GO, molecular_function)",1.23e-01,2
3,cell differentiation,"(GO, biological_process)",1.59e-01,3
4,anatomical structure formation involved in morphogenesis,"(GO, biological_process)",1.76e-01,2




***  C20 selected: 54 with EID: 54
Enrichment at FDR: 0.25 and min query - gene set overlap 2
N query genes in displayed gene sets: 4 out of 54 query genes used for enrichment calculation.


,Gene set,Ontology,FDR,N in query
0,Phagosome,"(KEGG, Pathways)",2.42e-01,2
1,decreased spore viability,"(Dictybase, Phenotypes)",2.42e-01,2




***  C21 selected: 45 with EID: 45
Enrichment at FDR: 0.25 and min query - gene set overlap 2
N query genes in displayed gene sets: 0 out of 45 query genes used for enrichment calculation.


None

#### Using only genes annotated with at least one gene set 
To find genes that have gene set annotations only gene sets used in enrichment are used (see above). Genes that are in at least one of these gene sets are used for enrichment calculation, both for reference and gene group of interest (query).

In [19]:
regulons=pd.read_table(pathRegulons+'mergedGenes_minExpressed0.990.1Strains1Min1Max18_clustersLouvain0.4minmaxNologPCA30kN30.tab',index_col=0)
for group in range(1,len(regulons['Cluster'].unique())+1):
    enriched=group_diff_enrichment(data=regulons,group='C'+str(group),min_overlap=2,use_annotated_genes=True)
    if  save_enrichment and enriched is not None:
        enriched.to_csv(pathRegulons+'enrichment/minExpressed0.990.1Strains1Min1Max18_clustersLouvain0.4minmaxNologPCA30kN30_annotatedOnly_enrichment_cluster'+str(group)+'.tsv', sep='\t',index=False)

***  C1 selected: 30 with EID: 30
Ratio of genes annotated with a gene set in reference 0.32 and query 0.7
Enrichment at FDR: 0.25 and min query - gene set overlap 2
N query genes in displayed gene sets: 20 out of 21 query genes used for enrichment calculation.


,Gene set,Ontology,FDR,N in query
0,Ribosome biogenesis in eukaryotes,"(KEGG, Pathways)",1.07e-15,12
1,nucleolus,"(GO, cellular_component)",8.54e-14,11
2,ribosome biogenesis,"(GO, biological_process)",6.14e-13,10
3,ribonucleoprotein complex assembly,"(GO, biological_process)",7.00e-06,4
4,RNA binding,"(GO, molecular_function)",5.82e-04,6
5,rRNA binding,"(GO, molecular_function)",5.45e-03,2
6,protein-containing complex assembly,"(GO, biological_process)",1.09e-02,4
7,cellular component assembly,"(GO, biological_process)",3.99e-02,4
8,helicase activity,"(GO, molecular_function)",5.26e-02,2
9,methyltransferase activity,"(GO, molecular_function)",5.62e-02,2




***  C2 selected: 66 with EID: 65
Ratio of genes annotated with a gene set in reference 0.32 and query 0.58
Enrichment at FDR: 0.25 and min query - gene set overlap 2
N query genes in displayed gene sets: 34 out of 38 query genes used for enrichment calculation.


,Gene set,Ontology,FDR,N in query
0,Oxidative phosphorylation,"(KEGG, Pathways)",9.24e-08,9
1,Metabolic pathways,"(KEGG, Pathways)",1.10e-04,19
2,Citrate cycle (TCA cycle),"(KEGG, Pathways)",5.02e-03,4
3,mitochondrion,"(GO, cellular_component)",5.37e-03,9
4,Phagosome,"(KEGG, Pathways)",2.56e-02,4
5,protein folding,"(GO, biological_process)",6.55e-02,3
6,aberrant cellular response to stress,"(Dictybase, Phenotypes)",6.55e-02,2
7,generation of precursor metabolites and energy,"(GO, biological_process)",6.72e-02,4
8,cytoplasmic vesicle,"(GO, cellular_component)",1.29e-01,6
9,Ribosome,"(KEGG, Pathways)",1.29e-01,4




***  C3 selected: 68 with EID: 68
Ratio of genes annotated with a gene set in reference 0.32 and query 0.97
Enrichment at FDR: 0.25 and min query - gene set overlap 2
N query genes in displayed gene sets: 62 out of 66 query genes used for enrichment calculation.


...

,Gene set,Ontology,FDR,N in query
0,extracellular region,"(GO, cellular_component)",2.26e-02,4
1,cell adhesion,"(GO, biological_process)",2.26e-02,3
2,cell differentiation,"(GO, biological_process)",4.39e-02,3
3,hydrolase activity,"(GO, molecular_function)",4.39e-02,2
4,anatomical structure formation involved in morphogenesis,"(GO, biological_process)",8.83e-02,2




***  C20 selected: 54 with EID: 54
Ratio of genes annotated with a gene set in reference 0.32 and query 0.2
Enrichment at FDR: 0.25 and min query - gene set overlap 2
N query genes in displayed gene sets: 6 out of 11 query genes used for enrichment calculation.


,Gene set,Ontology,FDR,N in query
0,Phagosome,"(KEGG, Pathways)",1.55e-01,2
1,decreased spore viability,"(Dictybase, Phenotypes)",1.55e-01,2
2,extracellular region,"(GO, cellular_component)",2.44e-01,2




***  C21 selected: 45 with EID: 45
Ratio of genes annotated with a gene set in reference 0.32 and query 0.13
Enrichment at FDR: 0.25 and min query - gene set overlap 2
N query genes in displayed gene sets: 3 out of 6 query genes used for enrichment calculation.


,Gene set,Ontology,FDR,N in query
0,cytosol,"(GO, cellular_component)",2.24e-01,2
1,response to stress,"(GO, biological_process)",2.24e-01,2


### Enrichment for mergedGenes_minExpressed0.990.1Strains1Min1Max18_clustersAX4Louvain0.4m0s1log.tab

In [13]:
regulons=pd.read_table(pathRegulons+'mergedGenes_minExpressed0.990.1Strains1Min1Max18_clustersAX4Louvain0.4m0s1log.tab',index_col=0)
for group in range(1,len(regulons['Cluster'].unique())+1):
    cluster='C'+str(group)
    print('*** Cluster %s (%s)' %(cluster,string.ascii_uppercase[group-1]))
    
    enriched=group_diff_enrichment(data=regulons,group=cluster,min_overlap=2)
    if  save_enrichment and enriched is not None:
        enriched.to_csv(pathRegulons+'enrichment/minExpressed0.990.1Strains1Min1Max18_clustersAX4Louvain0.4m0s1log_enrichment_cluster'+str(group)+'.tsv', sep='\t',index=False)

*** Cluster C1 (A)
***  C1 selected: 31 with EID: 31
Enrichment at FDR: 0.25 and min query - gene set overlap 2
N query genes in displayed gene sets: 20 out of 31 query genes used for enrichment calculation.


,Gene set,Ontology,FDR,N in query
0,Ribosome biogenesis in eukaryotes,"(KEGG, Pathways)",7.70e-19,12
1,nucleolus,"(GO, cellular_component)",9.74e-17,11
2,ribosome biogenesis,"(GO, biological_process)",1.15e-15,10
3,ribonucleoprotein complex assembly,"(GO, biological_process)",5.11e-07,4
4,RNA binding,"(GO, molecular_function)",1.41e-05,6
5,protein-containing complex assembly,"(GO, biological_process)",1.13e-03,4
6,rRNA binding,"(GO, molecular_function)",1.41e-03,2
7,cellular component assembly,"(GO, biological_process)",4.00e-03,4
8,helicase activity,"(GO, molecular_function)",1.68e-02,2
9,methyltransferase activity,"(GO, molecular_function)",1.82e-02,2




*** Cluster C2 (B)
***  C2 selected: 54 with EID: 54
Enrichment at FDR: 0.25 and min query - gene set overlap 2
N query genes in displayed gene sets: 26 out of 54 query genes used for enrichment calculation.


,Gene set,Ontology,FDR,N in query
0,Oxidative phosphorylation,"(KEGG, Pathways)",8.06e-08,7
1,mitochondrion,"(GO, cellular_component)",8.06e-08,11
2,Metabolic pathways,"(KEGG, Pathways)",1.08e-05,14
3,generation of precursor metabolites and energy,"(GO, biological_process)",1.18e-05,6
4,protein folding,"(GO, biological_process)",1.87e-04,4
5,unfolded protein binding,"(GO, molecular_function)",1.72e-03,3
6,oxidoreductase activity,"(GO, molecular_function)",1.88e-03,6
7,Phagosome,"(KEGG, Pathways)",9.03e-03,3
8,Citrate cycle (TCA cycle),"(KEGG, Pathways)",3.82e-02,2
9,Propanoate metabolism,"(KEGG, Pathways)",3.82e-02,2




*** Cluster C3 (C)
***  C3 selected: 54 with EID: 54
Enrichment at FDR: 0.25 and min query - gene set overlap 2
N query genes in displayed gene sets: 44 out of 54 query genes used for enrichment calculation.


...

,Gene set,Ontology,FDR,N in query
0,cell wall,"(GO, cellular_component)",1.83e-02,2
1,external encapsulating structure,"(GO, cellular_component)",1.83e-02,2
2,anatomical structure formation involved in morphogenesis,"(GO, biological_process)",4.02e-02,3
3,hydrolase activity,"(GO, molecular_function)",1.05e-01,2
4,cell differentiation,"(GO, biological_process)",1.20e-01,3
5,lipid binding,"(GO, molecular_function)",1.20e-01,2
6,extracellular region,"(GO, cellular_component)",1.25e-01,3




*** Cluster C19 (S)
***  C19 selected: 89 with EID: 88
Enrichment at FDR: 0.25 and min query - gene set overlap 2
N query genes in displayed gene sets: 10 out of 88 query genes used for enrichment calculation.


,Gene set,Ontology,FDR,N in query
0,extracellular region,"(GO, cellular_component)",2.53e-03,7
1,anatomical structure formation involved in morphogenesis,"(GO, biological_process)",1.90e-02,4
2,cell differentiation,"(GO, biological_process)",1.02e-01,4
3,Starch and sucrose metabolism,"(KEGG, Pathways)",1.02e-01,2




*** Cluster C20 (T)
***  C20 selected: 1 with EID: 1
Enrichment at FDR: 0.25 and min query - gene set overlap 2
N query genes in displayed gene sets: 0 out of 1 query genes used for enrichment calculation.


None

#### Using only genes annotated with at least one gene set

In [14]:
regulons=pd.read_table(pathRegulons+'mergedGenes_minExpressed0.990.1Strains1Min1Max18_clustersAX4Louvain0.4m0s1log.tab',index_col=0)
for group in range(1,len(regulons['Cluster'].unique())+1):
    cluster='C'+str(group)
    print('*** Cluster %s (%s)' %(cluster,string.ascii_uppercase[group-1]))

    enriched=group_diff_enrichment(data=regulons,group=cluster,min_overlap=2,use_annotated_genes=True)
    if save_enrichment and enriched is not None:
        enriched.to_csv(pathRegulons+'enrichment/mergedGenes_minExpressed0.990.1Strains1Min1Max18_clustersAX4Louvain0.4m0s1log.tab'+str(group)+'.tsv', sep='\t',index=False)

*** Cluster C1 (A)
***  C1 selected: 31 with EID: 31
Ratio of genes annotated with a gene set in reference 0.32 and query 0.71
Enrichment at FDR: 0.25 and min query - gene set overlap 2
N query genes in displayed gene sets: 20 out of 22 query genes used for enrichment calculation.


,Gene set,Ontology,FDR,N in query
0,Ribosome biogenesis in eukaryotes,"(KEGG, Pathways)",3.06e-15,12
1,nucleolus,"(GO, cellular_component)",2.21e-13,11
2,ribosome biogenesis,"(GO, biological_process)",1.46e-12,10
3,ribonucleoprotein complex assembly,"(GO, biological_process)",1.12e-05,4
4,RNA binding,"(GO, molecular_function)",1.01e-03,6
5,rRNA binding,"(GO, molecular_function)",7.85e-03,2
6,protein-containing complex assembly,"(GO, biological_process)",1.70e-02,4
7,cellular component assembly,"(GO, biological_process)",6.15e-02,4
8,helicase activity,"(GO, molecular_function)",7.53e-02,2
9,methyltransferase activity,"(GO, molecular_function)",8.04e-02,2




*** Cluster C2 (B)
***  C2 selected: 54 with EID: 54
Ratio of genes annotated with a gene set in reference 0.32 and query 0.57
Enrichment at FDR: 0.25 and min query - gene set overlap 2
N query genes in displayed gene sets: 26 out of 31 query genes used for enrichment calculation.


,Gene set,Ontology,FDR,N in query
0,Oxidative phosphorylation,"(KEGG, Pathways)",6.02e-06,7
1,mitochondrion,"(GO, cellular_component)",1.47e-05,11
2,generation of precursor metabolites and energy,"(GO, biological_process)",3.68e-04,6
3,protein folding,"(GO, biological_process)",2.08e-03,4
4,Metabolic pathways,"(KEGG, Pathways)",2.08e-03,14
5,unfolded protein binding,"(GO, molecular_function)",9.12e-03,3
6,oxidoreductase activity,"(GO, molecular_function)",3.29e-02,6
7,Phagosome,"(KEGG, Pathways)",4.52e-02,3
8,Citrate cycle (TCA cycle),"(KEGG, Pathways)",1.22e-01,2
9,Propanoate metabolism,"(KEGG, Pathways)",1.23e-01,2




*** Cluster C3 (C)
***  C3 selected: 54 with EID: 54
Ratio of genes annotated with a gene set in reference 0.32 and query 0.89
Enrichment at FDR: 0.25 and min query - gene set overlap 2
N query genes in displayed gene sets: 44 out of 48 query genes used for enrichment calculation.


...

,Gene set,Ontology,FDR,N in query
0,anatomical structure formation involved in morphogenesis,"(GO, biological_process)",3.13e-03,3
1,cell wall,"(GO, cellular_component)",3.13e-03,2
2,external encapsulating structure,"(GO, cellular_component)",3.13e-03,2
3,cell differentiation,"(GO, biological_process)",1.39e-02,3
4,extracellular region,"(GO, cellular_component)",1.39e-02,3
5,hydrolase activity,"(GO, molecular_function)",1.39e-02,2
6,lipid binding,"(GO, molecular_function)",2.02e-02,2
7,anatomical structure development,"(GO, biological_process)",2.87e-02,4
8,protein transport,"(GO, biological_process)",1.23e-01,2




*** Cluster C19 (S)
***  C19 selected: 89 with EID: 88
Ratio of genes annotated with a gene set in reference 0.32 and query 0.2
Enrichment at FDR: 0.25 and min query - gene set overlap 2
N query genes in displayed gene sets: 10 out of 18 query genes used for enrichment calculation.


,Gene set,Ontology,FDR,N in query
0,extracellular region,"(GO, cellular_component)",6.19e-05,7
1,anatomical structure formation involved in morphogenesis,"(GO, biological_process)",2.89e-03,4
2,cell differentiation,"(GO, biological_process)",2.41e-02,4
3,Starch and sucrose metabolism,"(KEGG, Pathways)",4.25e-02,2




*** Cluster C20 (T)
***  C20 selected: 1 with EID: 1
Ratio of genes annotated with a gene set in reference 0.32 and query 0.0
Enrichment at FDR: 0.25 and min query - gene set overlap 2
N query genes in displayed gene sets: 0 out of 0 query genes used for enrichment calculation.


None